# Exercise#2 : Question-Answering Task

Using a Bert model fine tuned for Q&A on Squad 2,0 dataset.

https://huggingface.co/distilbert-base-cased-distilled-squad


#### Google Colab
If you are running the code in Google colab, install the packages by uncommenting/running the cell below

In [1]:
# !pip install transformers torch

### Import appropriate model & config classes

In [1]:
# https://huggingface.co/docs/transformers/model_doc/auto#transformers.AutoModelForQuestionAnswering
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
import getpass


import torch

## 1. Auto Model Class for task

AutoModelForQuestionAnswering

https://huggingface.co/docs/transformers/tasks/question_answering

Model in use

https://huggingface.co/distilbert-base-cased-distilled-squad

In [2]:
# Model in use
model_name = "distilbert-base-cased-distilled-squad"

# Create tokenizer & model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

## 2. Setup context and questions

In [4]:
# Setup the context and questions

context = """Beyonce Giselle Knowles-Carter (/biː\ˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) 
is an American singer, songwriter, record producer and actress. Born and raised in Houston, 
Texas, she performed in various singing and dancing competitions as a child, and rose 
to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her 
father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of 
all time. Their hiatus saw the release of Beyonce\'s debut album, Dangerously in Love (2003), 
which established her as a solo artist worldwide, earned five Grammy Awards and featured the 
Billboard Hot 100 number-one singles \"Crazy in Love\" and \"Baby Boy.
"""

questions = [
   "When did Beyonce start becoming popular?",
   "When did Beyonce leave Destiny's Child and become a solo singer?",
   "What areas did Beyonce compete in when she was growing up?",
   "In what city and state did Beyonce grow up?",
   "In what R&B group was she the lead singer?"
]

## Demonstrates that model understands the context and is not just using pattern matching
## Uncomment to try these out !!!
# context = "two plus two is four. 6 plus 2 is eight. 1 plus nine is 10."
# questions = ["what is 2 + 2", "what is 6 + 2", "what is two + 1"]

## 3. Tokenize, and run inference

In [10]:
# Try for any question by changing index
index = 2
inputs = tokenizer(questions[index], context, return_tensors="pt")

outputs = model(**inputs)

# Printhe tokenized string
# print(inputs)
# print(tokenizer.decode(inputs['input_ids'][0]))

# print(type(outputs))

## 4. Interpret the output

https://huggingface.co/docs/transformers/v4.17.0/en/main_classes/output#transformers.modeling_outputs.QuestionAnsweringModelOutput
    
Output has the following tensors:

* **start_logits** A score for each token in the context. The token with the maximum score is the start token for the answer

* **end_logits** A score for each token in the context. The token with the maximum score is the start token for the answer


In [11]:
# Logits in the QuestionAnsweringOutput class are start/end of response
# LINK
start_scores = outputs.start_logits
end_scores = outputs.end_logits

# print(start_scores)
# print(end_scores)

# Get the start index = index for token that has the maximum score
start_index = torch.argmax(start_scores)

# This is to clarify the idea behind end index
print("Start index: ", start_index, "  Token at start_index: ", tokenizer.decode(inputs['input_ids'][0][start_index]))

# Get the end index = index for the token that has the maximum score
# Add 1 to include the last token
end_index = torch.argmax(end_scores) + 1

# This is to clarify the idea behind end index
print("End index: ", end_index, "  Token at end_index: ", tokenizer.decode(inputs['input_ids'][0][end_index-1]))

# Decode the tokens between start_index & end_index
answer = tokenizer.decode(inputs["input_ids"][0, start_index:end_index])

answer

Start index:  tensor(69)   Token at start_index:  singing
End index:  tensor(72)   Token at end_index:  dancing


'singing and dancing'

# Using pipeline

An instance of QuestionAnsweringPipeline is used. Refer to documentation for parameters.

https://huggingface.co/docs/transformers/v4.17.0/en/main_classes/pipelines#transformers.QuestionAnsweringPipeline

In [12]:
from transformers import pipeline

In [14]:
question_answerer_pipeline = pipeline("question-answering", model=model_name)

result = question_answerer_pipeline(question=questions[1],     context=context)
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Device set to use cpu


Answer: '2003', score: 0.9859, start: 532, end: 536


In [15]:
# https://huggingface.co/distilbert-base-cased-distilled-squad
from transformers import pipeline
question_answerer = pipeline("question-answering", model='distilbert-base-cased-distilled-squad')

question = "How many programming languages does BLOOM support?"
context = "BLOOM has 176 billion parameters and can generate text in 46 languages natural languages and 13 programming languages."

# https://huggingface.co/docs/transformers/v4.17.0/en/main_classes/pipelines#transformers.QuestionAnsweringPipeline
output = question_answerer(question=question, context=context)

print(output)


question = "How many parameters does BLOOM have?"
output = question_answerer(question=question, context=context)

print(output)

Device set to use cpu


{'score': 0.9425581097602844, 'start': 93, 'end': 95, 'answer': '13'}
{'score': 0.8635509014129639, 'start': 10, 'end': 21, 'answer': '176 billion'}


# Using Inference Client

Use the question_answering(...) function in InferenceClient class

https://huggingface.co/docs/huggingface_hub/v0.20.2/en/package_reference/inference_client#huggingface_hub.InferenceClient.question_answering



In [18]:
from huggingface_hub import InferenceClient
from dotenv import load_dotenv
import os

load_dotenv('E:\\Code\\gen-ai-app-dev-course\\.env')

True

In [29]:
# You will prompted for the HuggingFace token
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

client = InferenceClient(model="distilbert/distilbert-base-cased-distilled-squad" , token=HUGGINGFACEHUB_API_TOKEN)

In [30]:
client.question_answering(question,context)

QuestionAnsweringOutputElement(answer='176 billion', end=21, score=0.8635507225990295, start=10)